<a href="https://colab.research.google.com/github/EduardoRosa25/Trabalho_pratico_01_Grupo01/blob/main/Trabalho_Pratico_01_Grupo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alunos:
#  Bernardo Carvalho Trindade 12321BSI253
#  Eduardo Antonio 12311BSI317
#  Eduardo Rosa 12311BSI275
#  Luiz Fellipe 12311BSI262
#  Lucas Pinheiro Barbosa 12221BSI224

In [ ]:
# Certifique-se de que p arquivo_json = 'colecao - trabalho 01.json'" esteja na aba de arquivos do Colab

In [1]:
!pip install pandas numpy nltk

import json
import re
import sys
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from typing import List, Dict, Tuple, Set, Any
from collections import defaultdict
import zipfile

print("Ambiente configurado e bibliotecas importadas!")

Ambiente configurado e bibliotecas importadas!


In [2]:
# --- CLASSE 1: DOCUMENTO ---
class Documento:
    def __init__(self, doc_id: str, texto_original: str):
        self.id = doc_id
        self.texto_original = texto_original
        self.termos_processados: List[str] = []

# --- CLASSE 2: PROCESSADOR DE TEXTO ---
class ProcessadorTexto:
    def __init__(self):
        self._configurar_nltk()
        self.stemmer = RSLPStemmer()
        self.stopwords_pt = set(stopwords.words('portuguese'))

    def _configurar_nltk(self):
        try:
            nltk.data.find('corpora/stopwords')
            nltk.data.find('stemmers/rslp')
        except LookupError:
            print("[DEBUG] Baixando recursos do NLTK...")
            nltk.download('stopwords')
            nltk.download('rslp')

    def processar(self, texto: str) -> List[str]:
        texto = texto.lower()
        texto = re.sub(r'[^a-zà-úÀ-Ú\s]', '', texto)
        tokens = texto.split()

        tokens_processados = [
            self.stemmer.stem(t)
            for t in tokens
            if t not in self.stopwords_pt and len(t) > 1
        ]
        return tokens_processados

# --- CLASSE 3: ÍNDICE INVERTIDO (Com Busca por Frase) ---
class IndiceInvertido:
    def __init__(self):
        self.indice: Dict[str, Dict[str, List[int]]] = {}

    def construir_indice(self, documentos: Dict[str, Documento]):
        self.indice.clear()
        for doc_id, doc in documentos.items():
            for posicao, termo in enumerate(doc.termos_processados):
                if termo not in self.indice:
                    self.indice[termo] = {}
                if doc_id not in self.indice[termo]:
                    self.indice[termo][doc_id] = []
                self.indice[termo][doc_id].append(posicao)

    def get_indice(self) -> Dict[str, Dict[str, List[int]]]:
        return self.indice

    def exibir_indice(self):
        if not self.indice:
            print("[INFO] O Índice Invertido está vazio.")
            return
        print("\n--- Índice Invertido (Termo: {Doc_ID: [posições]}) ---")
        termos_ordenados = sorted(self.indice.keys())
        for termo in termos_ordenados:
            posting = self.indice[termo]
            formatado = " | ".join([f"{did}: {posting[did]}" for did in sorted(posting.keys())])
            print(f"'{termo}': {formatado}")

    def buscar_por_frase(self, frase_processada: List[str]) -> Set[str]:
        if not frase_processada:
            return set()

        termo_inicial = frase_processada[0]
        if termo_inicial not in self.indice:
            return set()

        candidatos = set()
        docs_do_termo_inicial = self.indice[termo_inicial]

        for doc_id, posicoes_iniciais in docs_do_termo_inicial.items():
            for pos_inicio in posicoes_iniciais:
                match_completo = True
                for i in range(1, len(frase_processada)):
                    termo_seguinte = frase_processada[i]
                    posicao_esperada = pos_inicio + i

                    if (termo_seguinte not in self.indice or
                        doc_id not in self.indice[termo_seguinte] or
                        posicao_esperada not in self.indice[termo_seguinte][doc_id]):
                        match_completo = False
                        break

                if match_completo:
                    candidatos.add(doc_id)
                    break
        return candidatos

# --- CLASSE 4: MATRIZ TF-IDF (Com Booleano e Vetorial) ---
class MatrizTFIDF:
    def __init__(self):
        self.vocabulario: List[str] = []
        self.matriz_tf: pd.DataFrame = pd.DataFrame()
        self.vetor_idf: pd.Series = pd.Series()
        self.matriz_tfidf: pd.DataFrame = pd.DataFrame()
        self.normas_doc: Dict[str, float] = {}
        self.processador = ProcessadorTexto()

    def construir_matrizes(self, documentos: Dict[str, Documento]):
        if not documentos:
            return

        nomes_docs = list(documentos.keys())
        docs_processados = [doc.termos_processados for doc in documentos.values()]

        self.vocabulario = sorted(set([t for doc in docs_processados for t in doc]))

        self.matriz_tf = pd.DataFrame(0.0, index=self.vocabulario, columns=nomes_docs)
        for nome, termos in zip(nomes_docs, docs_processados):
            for termo in termos:
                self.matriz_tf.loc[termo, nome] += 1

        self.matriz_tf = self.matriz_tf.map(lambda x: 1 + np.log2(x) if x > 0 else 0)

        N = len(nomes_docs)
        ni = (self.matriz_tf > 0).sum(axis=1)
        idf_vals = np.log2(N / ni.replace(0, 1))
        self.vetor_idf = pd.Series(idf_vals, index=self.vocabulario)

        self.matriz_tfidf = self.matriz_tf.mul(self.vetor_idf, axis=0)
        self.normas_doc = np.sqrt((self.matriz_tfidf**2).sum(axis=0)).to_dict()

    def exibir_matriz_tfidf(self):
        if self.matriz_tfidf.empty:
            print("[INFO] Matriz vazia.")
        else:
            print(self.matriz_tfidf.round(4))

    def buscar_booleana(self, query: str, operador: str) -> List[str]:
        if self.matriz_tfidf.empty: return []

        termos_query = self.processador.processar(query)
        termos_validos = [t for t in termos_query if t in self.matriz_tfidf.index]
        if not termos_validos: return []

        presenca = (self.matriz_tfidf.loc[termos_validos] > 0).astype(int)
        operador = operador.upper()

        if operador == 'AND':
            match = presenca.sum(axis=0) == len(termos_validos)
            return match[match].index.tolist()
        elif operador == 'OR':
            match = presenca.sum(axis=0) > 0
            return match[match].index.tolist()
        elif operador == 'NOT':
            match_pos = presenca.sum(axis=0) > 0
            docs_com_termo = match_pos[match_pos].index.tolist()
            todos = self.matriz_tfidf.columns.tolist()
            return list(set(todos) - set(docs_com_termo))
        return []

    def buscar_similaridade(self, query: str, indice: Dict) -> List[Tuple[str, float]]:
        if self.matriz_tfidf.empty: return []

        q_termos = self.processador.processar(query)
        q_tf = pd.Series([q_termos.count(t) for t in self.vocabulario], index=self.vocabulario)
        q_tf = q_tf.map(lambda x: 1 + np.log2(x) if x > 0 else 0)

        q_vec = q_tf * self.vetor_idf
        q_norm = np.sqrt((q_vec**2).sum())
        if q_norm == 0: return []

        candidatos = set()
        for t in q_termos:
            if t in indice:
                candidatos.update(indice[t].keys())

        resultados = []
        for doc_id in candidatos:
            doc_vec = self.matriz_tfidf[doc_id]
            dot = (q_vec * doc_vec).sum()
            d_norm = self.normas_doc.get(doc_id, 0)

            if d_norm > 0:
                score = dot / (q_norm * d_norm)
                if score > 0:
                    resultados.append((doc_id, score))

        return sorted(resultados, key=lambda x: x[1], reverse=True)

# --- CLASSE 5: COLEÇÃO  ---
class Colecao:
    def __init__(self):
        self.documentos: Dict[str, Documento] = {}
        self.vocabulario: List[str] = []
        self.fila_documentos: List[Dict[str, str]] = []
        self.lista_completa_backup: List[Dict[str, str]] = []

        # Instancia as classes que já definimos acima
        self.matriz_handler = MatrizTFIDF()
        self.processador = ProcessadorTexto()
        self.indice_invertido_handler = IndiceInvertido()

    def carregar_dados_iniciais(self, caminho_arquivo: str):
        if self.lista_completa_backup: return

        print(f"[DEBUG] Lendo JSON: {caminho_arquivo}")
        try:
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            dados_normalizados = []
            if isinstance(dados, list):
                dados_normalizados = dados
            elif isinstance(dados, dict):
                dados_normalizados = [{"name": k, "content": v} for k, v in dados.items()]

            self.lista_completa_backup = list(dados_normalizados)
            self.fila_documentos = list(dados_normalizados)
            print(f"[SUCESSO] {len(self.lista_completa_backup)} documentos carregados.")

        except FileNotFoundError:
            print(f"[ERRO] Arquivo '{caminho_arquivo}' não encontrado. Faça upload!")
        except Exception as e:
            print(f"[ERRO] Falha ao ler JSON: {e}")

    def adicionar_proximo_da_fila(self):
        if not self.fila_documentos:
            print("[AVISO] Fila vazia.")
            return

        item = self.fila_documentos.pop(0)
        d_id = item.get('name') or item.get('id')
        d_conteudo = item.get('content') or item.get('text')

        if d_id and d_conteudo:
            if d_id not in self.documentos:
                self._adicionar_documento_interno(d_id, d_conteudo)
                print(f"[SUCESSO] Documento {d_id} adicionado.")
            else:
                print(f"[INFO] Doc {d_id} já existe.")

    def adicionar_todos_restantes(self):
        if not self.lista_completa_backup:
            print("[ERRO] Carregue o JSON primeiro.")
            return

        print("Processando inserção em lote...")
        count = 0
        for item in self.lista_completa_backup:
            d_id = item.get('name') or item.get('id')
            d_conteudo = item.get('content') or item.get('text')

            if d_id and d_conteudo and d_id not in self.documentos:
                self._adicionar_documento_interno(d_id, d_conteudo, atualizar_agora=False)
                count += 1

        self.fila_documentos = []
        if count > 0:
            self._atualizar_sistema()
            print(f"[SUCESSO] {count} documentos adicionados.")
        else:
            print("[INFO] Todos os documentos já estavam presentes.")

    def remover_documento(self, doc_id: str):
        if doc_id in self.documentos:
            del self.documentos[doc_id]
            self._atualizar_sistema()
            print(f"[SUCESSO] Documento {doc_id} removido.")
        else:
            print("[ERRO] ID não encontrado.")

    def _adicionar_documento_interno(self, doc_id: str, texto: str, atualizar_agora=True):
        doc = Documento(doc_id, texto)
        doc.termos_processados = self.processador.processar(texto)
        self.documentos[doc_id] = doc

        if atualizar_agora:
            self._atualizar_sistema()

    def _atualizar_sistema(self):
        termos = set()
        for doc in self.documentos.values():
            termos.update(doc.termos_processados)
        self.vocabulario = sorted(list(termos))

        if self.documentos:
            self.matriz_handler.construir_matrizes(self.documentos)
            self.indice_invertido_handler.construir_indice(self.documentos)

    def get_indice_invertido_handler(self): return self.indice_invertido_handler
    def get_matriz_handler(self): return self.matriz_handler
    def get_vocabulario(self): return self.vocabulario

print("Todas as classes foram compiladas com sucesso!")

Todas as classes foram compiladas com sucesso!


In [3]:
def print_menu():
    print('\n=== MENU PRINCIPAL ===')
    print('1) Adicionar um documento por vez (fila)')
    print('2) Adicionar TODOS os documentos (lote)')
    print('3) Remover documento por ID')
    print('4) Exibir Vocabulário')
    print('5) Exibir Matriz TF-IDF')
    print('6) Exibir Índice Invertido (Posicional)')
    print('7) Consulta Booleana (AND, OR, NOT)')
    print('8) Consulta por Similaridade (Ranking)')
    print('9) Consulta por Frase (Exata)')
    print('0) Sair')

def main():
    sistema = Colecao()
    # Certifique-se de que este arquivo esteja na aba de arquivos do Colab
    arquivo_json = 'colecao - trabalho 01.json'

    sistema.carregar_dados_iniciais(arquivo_json)

    while True:
        print_menu()
        choice = input('Escolha uma opção: ').strip()

        try:
            if choice == '1':
                sistema.adicionar_proximo_da_fila()
            elif choice == '2':
                sistema.adicionar_todos_restantes()
            elif choice == '3':
                id_doc = input("Digite o ID do Documento (ex: D1): ").strip()
                sistema.remover_documento(id_doc)
            elif choice == '4':
                vocab = sistema.get_vocabulario()
                print(f"\nVocabulário ({len(vocab)} termos):")
                print(", ".join(vocab))
            elif choice == '5':
                sistema.get_matriz_handler().exibir_matriz_tfidf()
            elif choice == '6':
                sistema.get_indice_invertido_handler().exibir_indice()
            elif choice == '7':
                if not sistema.documentos:
                    print("[ERRO] Coleção vazia.")
                    continue
                print("\n--- Consulta Booleana ---")
                query = input("Termos da consulta: ")
                operador = input("Operador (AND, OR, NOT): ").strip()
                res = sistema.get_matriz_handler().buscar_booleana(query, operador)
                if res:
                    print(f"✅ Documentos encontrados: {', '.join(res)}")
                else:
                    print(f"❌ Nenhum resultado para '{query}' ({operador}).")
            elif choice == '8':
                if not sistema.documentos:
                    print("[ERRO] Coleção vazia.")
                    continue
                print("\n--- Consulta por Similaridade (Cosseno) ---")
                query = input("Digite a consulta: ")
                indice = sistema.get_indice_invertido_handler().get_indice()
                ranking = sistema.get_matriz_handler().buscar_similaridade(query, indice)
                if ranking:
                    print(f"✅ Ranking de Relevância:")
                    for doc_id, score in ranking:
                        print(f"1. {doc_id} (Score: {score:.4f})")
                else:
                    print("❌ Nenhum documento relevante encontrado.")
            elif choice == '9':
                if not sistema.documentos:
                    print("[ERRO] Coleção vazia.")
                    continue
                print("\n--- Consulta por Frase ---")
                frase = input("Digite a frase exata: ").strip()
                frase_processada = sistema.processador.processar(frase)
                if not frase_processada:
                    print("[AVISO] Frase inválida/vazia.")
                    continue
                res = sistema.get_indice_invertido_handler().buscar_por_frase(frase_processada)
                if res:
                    print(f"✅ Frase encontrada nos documentos: {', '.join(res)}")
                else:
                    print("❌ Frase não encontrada exatamente nessa ordem.")
            elif choice == '0':
                print("Encerrando...")
                break
            else:
                print("Opção inválida.")

        except Exception as e:
            print(f"[ERRO CRÍTICO] {e}")

if __name__ == '__main__':
    main()

[DEBUG] Baixando recursos do NLTK...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


[DEBUG] Lendo JSON: colecao - trabalho 01.json
[SUCESSO] 50 documentos carregados.

=== MENU PRINCIPAL ===
1) Adicionar um documento por vez (fila)
2) Adicionar TODOS os documentos (lote)
3) Remover documento por ID
4) Exibir Vocabulário
5) Exibir Matriz TF-IDF
6) Exibir Índice Invertido (Posicional)
7) Consulta Booleana (AND, OR, NOT)
8) Consulta por Similaridade (Ranking)
9) Consulta por Frase (Exata)
0) Sair
Escolha uma opção: 1
[SUCESSO] Documento D1 adicionado.

=== MENU PRINCIPAL ===
1) Adicionar um documento por vez (fila)
2) Adicionar TODOS os documentos (lote)
3) Remover documento por ID
4) Exibir Vocabulário
5) Exibir Matriz TF-IDF
6) Exibir Índice Invertido (Posicional)
7) Consulta Booleana (AND, OR, NOT)
8) Consulta por Similaridade (Ranking)
9) Consulta por Frase (Exata)
0) Sair
Escolha uma opção: 2
Processando inserção em lote...
[SUCESSO] 49 documentos adicionados.

=== MENU PRINCIPAL ===
1) Adicionar um documento por vez (fila)
2) Adicionar TODOS os documentos (lote)
3)

KeyboardInterrupt: Interrupted by user